<a href="https://colab.research.google.com/github/AntoniaAcevedo/ADA-Informe/blob/main/Informe11_ADA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Problema del camino mas corto
---


## 1.1 Bellman-Ford (version revisada)
---

**Entrada**: Un grafo dirigido $G=(V,E)$, un vértice fuente $s\in V$, y un valor real $l_e$ asociado a cada arco $e\in E$.

**Salida**: Una de las siguientes opciones:

1. La distancia más corta $dist(s,v)$ para cada vértice $v\in V$.
2. Una declaración indicando que $G$ contiene un ciclo negativo.

![imagen](https://www.researchgate.net/publication/282135528/figure/fig7/AS:280570845253639@1443904710246/The-schematic-procedure-of-the-Bellman-Ford-Moore-algorithm-other-related-explanations.png)

## 1.2 Dijkstra (arcos positivos)
---

**Entrada**: Un grafo dirigido $G=(V,E)$, un vértice fuente $s\in V$, y un valor real $l_e \geq 0$ asociado a cada arco $e\in E$.

**Salida**: La distancia más corta $dist(s,v)$ para cada vértice $v\in V$.

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT-ZKzHQ29pFBLLOx628CHa5v3i1tC5iAgASkoAJoqYf7K1oUmp" width="800" height="400">

# 2. Bellman-Ford y Dijkstra
---
El siguiente código muestra una implementación del algoritmo **Bellman-Ford** y la del algoritmo **Dijkstra**.

##2.1.1 Instancia (entera)
---

In [13]:
from termcolor import colored
import networkx as nx
import random

In [7]:
def is_valid_edge(generated_edges: dict, i: int, j: int):
    return i != j and not generated_edges.get((i, j), None) and not generated_edges.get((j, i), None)

In [8]:
def instance_entero(n: int):
    """
        Input: cantidad de vértices
        Output: una lista que contiene todos los arcos y el número del vértice fuente (la función retorna dos variables).
        Los arcos vienen en la forma (i, j, weight), donde i es el vértice origen del arco y j el vértice al que apunta el arco, mientras que weight es su peso.
    """
    graph = []
    nodes = random.sample(range(0, n), n)
    unvisited_nodes = random.sample(range(0, n), n)
    
    generated_edges = {}
    for i in nodes:
        rand = random.sample(nodes, random.randint(1, 3))

        for j in rand:
            edge = (i, j)
            edge_with_weight = (i, j, random.randint(1, 100))
            
            if generated_edges.get((edge[1], edge[0]), None):
                continue
            
            if i == j:
                new_vertice = None
                iterations = 0
                while new_vertice is None and iterations < 250:
                    iterations += 1
                    number = random.randint(0, n - 1)
                    if is_valid_edge(generated_edges, i, number):
                        new_vertice = number

                if iterations >= 250:
                    return instance_entero(n)
                
                edge = (i, new_vertice)
                edge_with_weight = (i, new_vertice, random.randint(-20, 100)) # -20 y 100 corresponde a los límites de los pesos, puede cambiarlos.
            
            graph.append(edge_with_weight)
            generated_edges[edge] = edge

            if edge_with_weight[1] in unvisited_nodes:
                unvisited_nodes.remove(edge_with_weight[1])

    for i in unvisited_nodes:
        valid_edge = False
        iterations = 0
        while not valid_edge and iterations < 250:
            iterations += 1
            m = random.randint(0, n - 1)
            if is_valid_edge(generated_edges, m, i):
                valid_edge = True
                edge = (m, i)
                edge_with_weight = (m, i, random.randint(-20, 100)) # -20 y 100 corresponde a los límites de los pesos, puede cambiarlos.
                graph.append(edge_with_weight)
                generated_edges[edge] = edge

        if iterations >= 250:
            return instance_entero(n)

    return graph, graph[0][0]

##2.1.2 Instancia (positiva)
---

In [9]:
def instance_pos(n: int):
    """
        Input: cantidad de vértices
        Output: una lista que contiene todos los arcos y el número del vértice fuente (la función retorna dos variables).
        Los arcos vienen en la forma (i, j, weight), donde i es el vértice origen del arco y j el vértice al que apunta el arco, mientras que weight es su peso.
    """
    graph = []
    nodes = random.sample(range(0, n), n)
    unvisited_nodes = random.sample(range(0, n), n)
    
    generated_edges = {}
    for i in nodes:
        rand = random.sample(nodes, random.randint(1, 3))

        for j in rand:
            edge = (i, j)
            edge_with_weight = (i, j, random.randint(1, 100))
            
            if generated_edges.get((edge[1], edge[0]), None):
                continue
            
            if i == j:
                new_vertice = None
                iterations = 0
                while new_vertice is None and iterations < 250:
                    iterations += 1
                    number = random.randint(0, n - 1)
                    if is_valid_edge(generated_edges, i, number):
                        new_vertice = number

                if iterations >= 250:
                    return instance_pos(n)
                
                edge = (i, new_vertice)
                edge_with_weight = (i, new_vertice, random.randint(1, 100)) # 1 y 100 corresponde a los límites de los pesos, puede cambiarlos.
            
            graph.append(edge_with_weight)
            generated_edges[edge] = edge

            if edge_with_weight[1] in unvisited_nodes:
                unvisited_nodes.remove(edge_with_weight[1])

    for i in unvisited_nodes:
        valid_edge = False
        iterations = 0
        while not valid_edge and iterations < 250:
            iterations += 1
            m = random.randint(0, n - 1)
            if is_valid_edge(generated_edges, m, i):
                valid_edge = True
                edge = (m, i)
                edge_with_weight = (m, i, random.randint(1, 100)) # 1 y 100 corresponde a los límites de los pesos, puede cambiarlos.
                graph.append(edge_with_weight)
                generated_edges[edge] = edge

        if iterations >= 250:
            return instance_pos(n)

    return graph, graph[0][0]

## 2.2 Codigo
---

### 2.2.1 Bellman-Ford
---

In [31]:
class Graph:
 
    def __init__(self, vertices):
        self.V = vertices 
        self.graph = []

    def addEdge(self, u, v, w):
        self.graph.append([u, v, w])

    def printArr(self, dist):
        print("Vertex Distance from Source")
        for i in range(self.V):
            print("{0}\t\t{1}".format(i, dist[i]))

    def BellmanFord(self, src, verbose=False):
        dist = [float("Inf")] * self.V
        dist[src] = 0
        for i in range(self.V - 1):
            for u, v, w in self.graph:
                if dist[u] != float("Inf") and dist[u] + w < dist[v]:
                    dist[v] = dist[u] + w
            if verbose:
              print("Iteracion", str(i + 1))
              print(colored(f"{str(dist)}", "blue"))
        if verbose:
          print("Busqueda de ciclos negativos")
        for u, v, w in self.graph:
            if dist[u] != float("Inf") and dist[u] + w < dist[v]:
                print("El grafo contiene ciclos negativos.")
                dist = list()
                return dist

        self.printArr(dist)
 

In [11]:
list_g,raiz = instance_entero(4)
grafo = Graph(4)
for (u,v,peso) in list_g:
  grafo.addEdge(u,v,peso)

grafo.BellmanFord(raiz)


Vertex Distance from Source
0		23
1		77
2		0
3		16


### 2.2.2 Dijkstra
---

In [12]:
class Graph_D():
 
    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[0 for column in range(vertices)]
                      for row in range(vertices)]
 
    def printSolution(self, dist):
        print("Vertex \t Distance from Source")
        for node in range(self.V):
            print(node, "\t\t", dist[node])
    def minDistance(self, dist, sptSet):
        min = 1e7
        for v in range(self.V):
            if dist[v] < min and sptSet[v] == False:
                min = dist[v]
                min_index = v
 
        return min_index

    def dijkstra(self, src):
 
        dist = [1e7] * self.V
        dist[src] = 0
        sptSet = [False] * self.V
 
        for cout in range(self.V):
            u = self.minDistance(dist, sptSet)
            sptSet[u] = True
            for v in range(self.V):
                if (self.graph[u][v] > 0 and
                   sptSet[v] == False and
                   dist[v] > dist[u] + self.graph[u][v]):
                    dist[v] = dist[u] + self.graph[u][v]
 
        self.printSolution(dist)
 
# Driver program
g = Graph(9)
g.graph = [[0, 4, 0, 0, 0, 0, 0, 8, 0],
           [4, 0, 8, 0, 0, 0, 0, 11, 0],
           [0, 8, 0, 7, 0, 4, 0, 0, 2],
           [0, 0, 7, 0, 9, 14, 0, 0, 0],
           [0, 0, 0, 9, 0, 10, 0, 0, 0],
           [0, 0, 4, 14, 10, 0, 2, 0, 0],
           [0, 0, 0, 0, 0, 2, 0, 1, 6],
           [8, 11, 0, 0, 0, 0, 1, 0, 7],
           [0, 0, 2, 0, 0, 0, 6, 7, 0]
           ]
 
g.dijkstra(0)
 

AttributeError: ignored

## 2.3 Visualizacion
---

In [ ]:
def graph_to_nxdigraph(graph: list, n: int):
    """
        Input: Un grafo en formato list[tuple]. Ej: [(0, 1, 10), (1, 2, 15), (2, 0, 7)].
        Output: Un nx.DiGraph de la libreria networkx.
    """
    nxdigraph = nx.DiGraph()
    [nxdigraph.add_node(i) for i in range(n)]

    for v in graph:
        nxdigraph.add_edge(v[0], v[1], weight=v[2])

    return nxdigraph

##2.4 Descripcion del algoritmo
---

### 2.4.1 Bellman-Ford
---

1. Inicializa un arreglo de distancia en infinito, exceptuando la del nodo inicial que es 0.
2. Se realiza el proceso de relajacion `N-1` veces, donde `N` es la cantidad de nodos.
3. Comprueba la existencia de ciclos negativos.
4. Entrega el camino mas corto entre la posicion inicial y todos los nodos, o un mensaje indicando la presencia de un ciclo negativo. 



### 2.4.2 Dijkstra
---

##2.5 Ejemplo
---

##2.6.1 Ejecucion paso a paso funcion Bellman-Ford (verbose=True)
---

In [43]:
list_g,raiz = instance_entero(13)
grafo = Graph(13)
for (u,v,peso) in list_g:
  grafo.addEdge(u,v,peso)

grafo.BellmanFord(raiz,True)

Iteracion 1
[7, inf, 70, 23, 92, 20, inf, inf, inf, inf, 59, inf, 0]
Iteracion 2
[7, 116, 70, 23, 92, 20, 115, 93, 213, 94, 59, 172, 0]
Iteracion 3
[7, 116, 70, 23, 92, 20, 115, 93, 213, 94, 59, 172, 0]
Iteracion 4
[7, 116, 70, 23, 92, 20, 115, 93, 213, 94, 59, 172, 0]
Iteracion 5
[7, 116, 70, 23, 92, 20, 115, 93, 213, 94, 59, 172, 0]
Iteracion 6
[7, 116, 70, 23, 92, 20, 115, 93, 213, 94, 59, 172, 0]
Iteracion 7
[7, 116, 70, 23, 92, 20, 115, 93, 213, 94, 59, 172, 0]
Iteracion 8
[7, 116, 70, 23, 92, 20, 115, 93, 213, 94, 59, 172, 0]
Iteracion 9
[7, 116, 70, 23, 92, 20, 115, 93, 213, 94, 59, 172, 0]
Iteracion 10
[7, 116, 70, 23, 92, 20, 115, 93, 213, 94, 59, 172, 0]
Iteracion 11
[7, 116, 70, 23, 92, 20, 115, 93, 213, 94, 59, 172, 0]
Iteracion 12
[7, 116, 70, 23, 92, 20, 115, 93, 213, 94, 59, 172, 0]
Busqueda de ciclos negativos
Vertex Distance from Source
0		7
1		116
2		70
3		23
4		92
5		20
6		115
7		93
8		213
9		94
10		59
11		172
12		0


##2.6.2 Ejecucion paso a paso funcion Dijkstra (verbose=True)
---

#3. Tiempo de ejecución
---

### 3.1 Bellman-Ford
---

### 3.2 Dijkstra
---

#4. Correctitud

---

### 4.1 Bellman-Ford
---

### **Teorema *(Correctitud)***

Se garantiza que si un grafo `G[V,E]` no contiene ciclos negativos, el algoritmo calculo correctamente la  distancia `D[S,v]` a cada nodo `v` en `V`.

### **Propiedad invariante**

Para cada nodo `v`, la distancia conocida visitados `j` arcos es minima.



```
j <= v-1:
  relajar arco: []
else:
  ciclo[]
``` 



### 4.2 Dijkstra
---